In [1]:
import pandas as pd
import numpy as np
from datetime import date 

In [2]:
# read in ppe info from ca open data portal
ppe = pd.read_csv("../../datasets_large/COVID-19_PPE_Logistics/logistics_ppe.csv")

In [3]:
ppe = ppe.loc[ppe["as_of_date"] >= '2021-02-27']

In [4]:
# nan values?
ppe.count()

county                      1043412
product_family              1043104
quantity_filled              934235
shipping_zip_postal_code     963637
as_of_date                  1043412
dtype: int64

In [5]:
# drop rows with nan values for the following columns
ppe = ppe.dropna(subset=["quantity_filled", "county", "product_family"])

In [6]:
# dates are all over the place, sorting from earliest to latest (low to high)
ppe = ppe.sort_values("as_of_date")

In [7]:
# explore what is in the 'county' column
print(ppe["county"].unique())
print(len(ppe["county"].unique()))

['State Agency' 'Los Angeles' 'Solano' 'Non-Governmental Entity' 'Fresno'
 'Mendocino' 'Trinity' 'Nevada' 'Inyo' 'Yolo' 'Santa Cruz' 'Imperial'
 'Marin' 'San Bernardino' 'Kern' 'Orange' 'Sutter' 'Tulare' 'Siskiyou'
 'Alameda' 'Stanislaus' 'Lake' 'Del Norte' 'San Mateo' 'Tehama'
 'San Francisco' 'San Joaquin' 'Santa Barbara' 'Sacramento' 'Madera'
 'Yuba' 'Santa Clara' 'Humboldt' 'San Diego' 'San Benito' 'Kings'
 'Mariposa' 'Riverside' 'San Luis Obispo' 'Modoc' 'Calaveras' 'Monterey'
 'Tuolumne' 'Napa' 'Ventura' 'Tribal' 'El Dorado' 'Placer' 'Contra Costa'
 'Colusa' 'Sonoma' 'Amador' 'Mono' 'Shasta' 'Lassen' 'Butte' 'Merced'
 'Glenn' 'Plumas' 'Sierra' 'Alpine' 'Other' 'State Agency or Other'
 'Unassigned']
64


In [8]:
# include only county specific data
clean_county = ppe.loc[ (ppe["county"] != "Non-Governmental Entity") & (ppe["county"] != "Governmental Entity") & (ppe["county"] != "State Agency") & (ppe["county"] != "Tribal") & (ppe["county"] != "State Agency or Other") & (ppe["county"] != "Unassigned") & (ppe["county"] != "Other") & (ppe["as_of_date"] != "2020-06-08") & (ppe["as_of_date"] != "2020-06-09")]

In [9]:
# check our results and create list of counties
counties = clean_county["county"].unique().tolist()
print(counties)
print(len(counties))

['Los Angeles', 'Solano', 'Fresno', 'Mendocino', 'Trinity', 'Nevada', 'Inyo', 'Yolo', 'Santa Cruz', 'Imperial', 'Marin', 'San Bernardino', 'Kern', 'Orange', 'Sutter', 'Tulare', 'Siskiyou', 'Alameda', 'Stanislaus', 'Lake', 'Del Norte', 'San Mateo', 'Tehama', 'San Francisco', 'San Joaquin', 'Santa Barbara', 'Sacramento', 'Madera', 'Yuba', 'Santa Clara', 'Humboldt', 'San Diego', 'San Benito', 'Kings', 'Mariposa', 'Riverside', 'San Luis Obispo', 'Modoc', 'Calaveras', 'Monterey', 'Tuolumne', 'Napa', 'Ventura', 'El Dorado', 'Placer', 'Contra Costa', 'Colusa', 'Sonoma', 'Amador', 'Mono', 'Shasta', 'Lassen', 'Butte', 'Merced', 'Glenn', 'Plumas', 'Sierra', 'Alpine']
58


In [10]:
# all dates placed into a new list
dates = clean_county["as_of_date"].unique().tolist()

# create list of index numbers to find weekly dates
weeks = np.arange(0, len(dates), 7).tolist()

In [11]:
# look at our lists for quality assurance
print(dates)
print(weeks)

['2021-02-27', '2021-02-28', '2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05', '2021-03-06', '2021-03-07', '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11', '2021-03-12', '2021-03-13', '2021-03-14', '2021-03-15', '2021-03-16', '2021-03-17', '2021-03-18', '2021-03-19', '2021-03-20']
[0, 7, 14, 21]


In [12]:
# create a test list of product families
p_list = clean_county["product_family"].unique().tolist()
print(p_list)
print(len(p_list))

['Examination Gloves', 'Viral Testing Media', 'Swabs', 'Other / None of the above', 'Cleaning Supplies', 'Surgical or Examination Gowns', 'Wipes', 'Shoe Covers', 'Surgical Masks', 'Face Shields (Disposable)', 'Hand Sanitizers', 'Pharmaceuticals', 'Cloth Masks', 'Lab Supplies', 'Coveralls (Hospitals or EMS)', 'Medical Equipment', 'N-95 Respirators', 'KN95 Respirators', 'Test Kits', 'Sample Collecting Kits', 'Goggles', 'Lab Kit', 'Body Bags', 'Personnel', 'Beds/Cots', 'Ventilators', 'Lab Platform', 'Fit Testing Supplies', 'Oxygen Distribution (manifolds, hoses, etc.)', 'Vaccine Supplies', 'Coroner Supplies', 'Oxygen Concentrator Accessories', 'Vaccine Personnel', 'Bedding Accessories', 'Oxygen Concentrators']
35


In [13]:
# new list of products to add to old list
old_list = ['Face Shields (Disposable)', 'Surgical Masks', 'Cloth Masks', 'Surgical or Examination Gowns',
            'Examination Gloves', 'N-95 Respirators', 'Hand Sanitizers', 'Wipes', 'Other / None of the above',
            'Cleaning Supplies', 'Swabs', 'Viral Testing Media', 'Pharmaceuticals', 'Shoe Covers', 'Test Kits',
            'KN95 Respirators', 'Coveralls (Hospitals or EMS)', 'Medical Equipment', 'Goggles', 'Lab Supplies',
            'Beds/Cots', 'Body Bags', 'Lab Kit', 'Ventilators', 'Personnel', 'Sample Collecting Kits', 'Lab Platform',
            'Bedding Accessories', 'Fit Testing Supplies']
new_list = []
for x in p_list:
    if x not in old_list:
        new_list.append(x)
print(new_list)
print(len(new_list))

['Oxygen Distribution (manifolds, hoses, etc.)', 'Vaccine Supplies', 'Coroner Supplies', 'Oxygen Concentrator Accessories', 'Vaccine Personnel', 'Oxygen Concentrators']
6


In [14]:
# include only county specific data
clean_county = clean_county.loc[(clean_county["product_family"] != "Oxygen Concentrator Accessories") & (clean_county["product_family"] != "Vaccine Supplies")  & (clean_county["product_family"] != "Oxygen Distribution (manifolds, hoses, etc.)") & (clean_county["product_family"] != "Coroner Supplies") & (clean_county["product_family"] != "Vaccine Personnel") & (clean_county["product_family"] != "Oxygen Concentrators")]

In [15]:
# create new list of product families
product_list = clean_county["product_family"].unique().tolist()
print(product_list)
print(len(product_list))

['Examination Gloves', 'Viral Testing Media', 'Swabs', 'Other / None of the above', 'Cleaning Supplies', 'Surgical or Examination Gowns', 'Wipes', 'Shoe Covers', 'Surgical Masks', 'Face Shields (Disposable)', 'Hand Sanitizers', 'Pharmaceuticals', 'Cloth Masks', 'Lab Supplies', 'Coveralls (Hospitals or EMS)', 'Medical Equipment', 'N-95 Respirators', 'KN95 Respirators', 'Test Kits', 'Sample Collecting Kits', 'Goggles', 'Lab Kit', 'Body Bags', 'Personnel', 'Beds/Cots', 'Ventilators', 'Lab Platform', 'Fit Testing Supplies', 'Bedding Accessories']
29


In [16]:
# how many weeks do we have?
print(len(weeks))

4


In [17]:
# create lists of 7 day periods
periods = []
for ref in weeks:
    try:
        period = dates[ref:ref+7]
        periods.append(period)
    except:
        print(dates[ref])

In [18]:
print(periods)

[['2021-02-27', '2021-02-28', '2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05'], ['2021-03-06', '2021-03-07', '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11', '2021-03-12'], ['2021-03-13', '2021-03-14', '2021-03-15', '2021-03-16', '2021-03-17', '2021-03-18', '2021-03-19'], ['2021-03-20']]


In [19]:
# reset the index for our big for loop later
clean_county = clean_county.reset_index(drop=True)

In [20]:
# check our results
clean_county

,county,product_family,quantity_filled,shipping_zip_postal_code,as_of_date
0,Los Angeles,Examination Gloves,10000.0,90806,2021-02-27
1,Los Angeles,Examination Gloves,10000.0,90815,2021-02-27
2,Los Angeles,Examination Gloves,10000.0,90815,2021-02-27
3,Solano,Viral Testing Media,4000.0,94533,2021-02-27
4,Solano,Swabs,10000.0,94533,2021-02-27
...,...,...,...,...,...
621138,Sonoma,Face Shields (Disposable),200.0,95492,2021-03-20
621139,Orange,Hand Sanitizers,12.0,92805,2021-03-20
621140,Orange,Face Shields (Disposable),200.0,92805,2021-03-20
621141,Orange,Surgical Masks,16000.0,92805,2021-03-20


In [21]:
# create list of empty columns
column_names = ["county", "week", "week number"]
# add a new empty column to our list from the product_list
for product in product_list:
    column_names.append(product)
# create dateframe with empty columns
weekly_df = pd.DataFrame(columns = column_names)
weekly_df

,county,week,week number,Examination Gloves,Viral Testing Media,Swabs,Other / None of the above,Cleaning Supplies,Surgical or Examination Gowns,Wipes,...,Sample Collecting Kits,Goggles,Lab Kit,Body Bags,Personnel,Beds/Cots,Ventilators,Lab Platform,Fit Testing Supplies,Bedding Accessories


In [22]:
# use a dictionary to create multiple empty lists to store weekly quantity filled
obj = {}
for product in product_list:
    obj[product] = []
print(obj)

{'Examination Gloves': [], 'Viral Testing Media': [], 'Swabs': [], 'Other / None of the above': [], 'Cleaning Supplies': [], 'Surgical or Examination Gowns': [], 'Wipes': [], 'Shoe Covers': [], 'Surgical Masks': [], 'Face Shields (Disposable)': [], 'Hand Sanitizers': [], 'Pharmaceuticals': [], 'Cloth Masks': [], 'Lab Supplies': [], 'Coveralls (Hospitals or EMS)': [], 'Medical Equipment': [], 'N-95 Respirators': [], 'KN95 Respirators': [], 'Test Kits': [], 'Sample Collecting Kits': [], 'Goggles': [], 'Lab Kit': [], 'Body Bags': [], 'Personnel': [], 'Beds/Cots': [], 'Ventilators': [], 'Lab Platform': [], 'Fit Testing Supplies': [], 'Bedding Accessories': []}


In [23]:
# figuring out how to add dictionaries to another
dict1 = {'key1': 'for', 'key2': 'geeks'}
dict2 = {'key3': 'for', 'key4': 'geeks'}
dict1.update(dict2)
dict_df = pd.DataFrame([dict1])
dict_df

,key1,key2,key3,key4
0,for,geeks,for,geeks


In [24]:
clean_county.loc[(clean_county["county"] == "Kern") & (clean_county["product_family"] == product_list[0])]

,county,product_family,quantity_filled,shipping_zip_postal_code,as_of_date
65,Kern,Examination Gloves,3000.0,93305,2021-02-27
3539,Kern,Examination Gloves,2000.0,93301,2021-02-27
5151,Kern,Examination Gloves,60000.0,93307,2021-02-27
5152,Kern,Examination Gloves,60000.0,93307,2021-02-27
5153,Kern,Examination Gloves,41000.0,93307,2021-02-27
...,...,...,...,...,...
618819,Kern,Examination Gloves,171000.0,93215,2021-03-20
619793,Kern,Examination Gloves,60000.0,93307,2021-03-20
619795,Kern,Examination Gloves,41000.0,93307,2021-03-20
619796,Kern,Examination Gloves,60000.0,93307,2021-03-20


In [25]:
test_weeks = [['2020-12-05', '2020-12-06', '2020-12-07', '2020-12-08', '2020-12-09', '2020-12-10', '2020-12-11'],
              ['2020-12-12', '2020-12-13', '2020-12-14', '2020-12-15', '2020-12-16', '2020-12-17', '2020-12-18'],
              ['2020-12-19', '2020-12-20', '2020-12-21', '2020-12-22', '2020-12-23', '2020-12-24', '2020-12-25']]
for ref in test_weeks:
    print(ref[0])
    print(ref[6])

2020-12-05
2020-12-11
2020-12-12
2020-12-18
2020-12-19
2020-12-25


In [26]:
# test loop just to see if things work
counter = 1
test_weeks = [['2020-11-14', '2020-11-15', '2020-11-16', '2020-11-17', '2020-11-18', '2020-11-19', '2020-11-20'],
              ['2020-11-21', '2020-11-22', '2020-11-23', '2020-11-24', '2020-11-25', '2020-11-26', '2020-11-27'],
              ['2020-11-28', '2020-11-29', '2020-11-30', '2020-12-01', '2020-12-02', '2020-12-03', '2020-12-04']]
test_counties = ['Kern', 'Mariposa', 'Merced']
test_list = ['Face Shields (Disposable)', 'Surgical Masks', 'Cloth Masks']

for ref in test_weeks:
    for county in test_counties:
        # use a dictionary to create multiple empty lists
        obj = {}
        for product in test_list:
            obj[product] = []
        for x in range(len(test_list)):
            try:
                df = clean_county.loc[(clean_county["county"] == county) & (clean_county["product_family"] == test_list[x])]
                # reset the index, get the correct dates by index and the total quantity filled
                df = df.reset_index(drop=True)
                df = df.loc[(df["as_of_date"] >= ref[0]) & (df["as_of_date"] <= ref[6])]
                weekly_filled = df["quantity_filled"].sum()
                obj[test_list[x]].append(weekly_filled)
                period = f"{ref[0]} to {ref[6]}"
                print(f"For {county} County, week {counter}, ppe item {x+1}:{test_list[x]}")
            except:
                print("Not enough dates")
        
        main_dict = {'county': [county], 'week': period, 'week number': counter}
        main_dict.update(obj)
        data = pd.DataFrame([main_dict])
        weekly_df = weekly_df.append(data)
    counter += 1

For Kern County, week 1, ppe item 1:Face Shields (Disposable)
For Kern County, week 1, ppe item 2:Surgical Masks
For Kern County, week 1, ppe item 3:Cloth Masks
For Mariposa County, week 1, ppe item 1:Face Shields (Disposable)
For Mariposa County, week 1, ppe item 2:Surgical Masks
For Mariposa County, week 1, ppe item 3:Cloth Masks
For Merced County, week 1, ppe item 1:Face Shields (Disposable)
For Merced County, week 1, ppe item 2:Surgical Masks
For Merced County, week 1, ppe item 3:Cloth Masks
For Kern County, week 2, ppe item 1:Face Shields (Disposable)
For Kern County, week 2, ppe item 2:Surgical Masks
For Kern County, week 2, ppe item 3:Cloth Masks
For Mariposa County, week 2, ppe item 1:Face Shields (Disposable)
For Mariposa County, week 2, ppe item 2:Surgical Masks
For Mariposa County, week 2, ppe item 3:Cloth Masks
For Merced County, week 2, ppe item 1:Face Shields (Disposable)
For Merced County, week 2, ppe item 2:Surgical Masks
For Merced County, week 2, ppe item 3:Cloth Mask

In [27]:
weekly_df

,county,week,week number,Examination Gloves,Viral Testing Media,Swabs,Other / None of the above,Cleaning Supplies,Surgical or Examination Gowns,Wipes,...,Sample Collecting Kits,Goggles,Lab Kit,Body Bags,Personnel,Beds/Cots,Ventilators,Lab Platform,Fit Testing Supplies,Bedding Accessories
0,[Kern],2020-11-14 to 2020-11-20,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,[Mariposa],2020-11-14 to 2020-11-20,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,[Merced],2020-11-14 to 2020-11-20,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,[Kern],2020-11-21 to 2020-11-27,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,[Mariposa],2020-11-21 to 2020-11-27,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,[Merced],2020-11-21 to 2020-11-27,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,[Kern],2020-11-28 to 2020-12-04,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,[Mariposa],2020-11-28 to 2020-12-04,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,[Merced],2020-11-28 to 2020-12-04,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# create list of empty columns
column_names = ["county", "week", "week number"]
# add a new empty column to our list from the product_list
for product in product_list:
    column_names.append(product)
# create dateframe with empty columns
weekly_df = pd.DataFrame(columns = column_names)
weekly_df

,county,week,week number,Examination Gloves,Viral Testing Media,Swabs,Other / None of the above,Cleaning Supplies,Surgical or Examination Gowns,Wipes,...,Sample Collecting Kits,Goggles,Lab Kit,Body Bags,Personnel,Beds/Cots,Ventilators,Lab Platform,Fit Testing Supplies,Bedding Accessories


In [29]:
# loop through each week, calculate the total quantity filled that week per product, per county..
# and append to our empty dataframe; will take a long time!
counter = 1
for ref in periods:
    for county in counties:
        # use a dictionary to create multiple empty lists
        obj = {}
        for product in product_list:
            obj[product] = []
        for x in range(len(product_list)):
            try:
                df = clean_county.loc[(clean_county["county"] == county) & (clean_county["product_family"] == product_list[x])]
                # reset the index, get the correct dates by index and the total quantity filled
                df = df.reset_index(drop=True)
                df = df.loc[(df["as_of_date"] >= ref[0]) & (df["as_of_date"] <= ref[6])]
                weekly_filled = df["quantity_filled"].sum()
                obj[product_list[x]].append(weekly_filled)
                period = f"{ref[0]} to {ref[6]}"
                # print(f"For {county} County, week {counter}, ppe item {x+1}:{product_list[x]}")
            except:
                print("Not enough dates")
        main_dict = {'county': [county], 'week': period, 'week number': counter}
        main_dict.update(obj)
        data = pd.DataFrame([main_dict])
        weekly_df = weekly_df.append(data)
    counter += 1
    try:
        print(f"Processed week from {ref[0]} through {ref[6]}...")
    except:
        print("Done.")

Processed week from 2021-02-27 through 2021-03-05...
Processed week from 2021-03-06 through 2021-03-12...
Processed week from 2021-03-13 through 2021-03-19...
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enou

Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dat

Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dat

Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dates
Not enough dat

In [30]:
# new and improved dataframe, ready for charts and presentation
weekly_df

,county,week,week number,Examination Gloves,Viral Testing Media,Swabs,Other / None of the above,Cleaning Supplies,Surgical or Examination Gowns,Wipes,...,Sample Collecting Kits,Goggles,Lab Kit,Body Bags,Personnel,Beds/Cots,Ventilators,Lab Platform,Fit Testing Supplies,Bedding Accessories
0,[Los Angeles],2021-02-27 to 2021-03-05,1,[411543000.0],[7616182.0],[11036550.0],[1773730.0],[170800.0],[88477892.0],[27608472.0],...,[385000.0],[480600.0],[0.0],[33740.0],[24894.0],[1834.0],[7392.0],[0.0],[266.0],[0.0]
0,[Solano],2021-02-27 to 2021-03-05,1,[17705520.0],[411978.0],[693700.0],[100457.0],[0.0],[678496.0],[40320.0],...,[0.0],[70700.0],[45500.0],[0.0],[126.0],[0.0],[0.0],[0.0],[35.0],[0.0]
0,[Fresno],2021-02-27 to 2021-03-05,1,[34865040.0],[801500.0],[1052800.0],[491519.0],[0.0],[1321810.0],[1820000.0],...,[10500.0],[79800.0],[0.0],[0.0],[2541.0],[0.0],[28.0],[0.0],[861.0],[0.0]
0,[Mendocino],2021-02-27 to 2021-03-05,1,[3920000.0],[122241.0],[207200.0],[80150.0],[224000.0],[507710.0],[266560.0],...,[68152.0],[21000.0],[0.0],[0.0],[84.0],[301.0],[21.0],[0.0],[0.0],[0.0]
0,[Trinity],2021-02-27 to 2021-03-05,1,[1750000.0],[20650.0],[35707.0],[18200.0],[0.0],[89978.0],[25200.0],...,[17500.0],[0.0],[0.0],[0.0],[56.0],[0.0],[0.0],[0.0],[0.0],[0.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,[Merced],2021-03-13 to 2021-03-19,4,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
0,[Glenn],2021-03-13 to 2021-03-19,4,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
0,[Plumas],2021-03-13 to 2021-03-19,4,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
0,[Sierra],2021-03-13 to 2021-03-19,4,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]


In [32]:
# read in ppe info from previous ppe_cruncher notebook
ppe_crunched = pd.read_csv("resources/ppe_crunched.csv")

In [33]:
# append new dataframe to previous ppe info, for a complete dataset
updated_df = ppe_crunched.append(weekly_df)

In [34]:
# export dataframe of results with today's date
today = date.today()
d_today = today.strftime("%m_%d_%Y")

In [35]:
updated_df

,county,week,week number,Face Shields (Disposable),Surgical Masks,Cloth Masks,Surgical or Examination Gowns,Examination Gloves,N-95 Respirators,Hand Sanitizers,...,Lab Supplies,Beds/Cots,Body Bags,Lab Kit,Ventilators,Personnel,Sample Collecting Kits,Lab Platform,Bedding Accessories,Fit Testing Supplies
0,['Kern'],2020-06-10 to 2020-06-16,1,[1656500.0],[2282000.0],[3673500.0],[2143190.0],[1161000.0],[3658718.0],[5707127.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[539.0],[0.0],[0.0],[0.0],[0.0]
1,['Mariposa'],2020-06-10 to 2020-06-16,1,[840.0],[568000.0],[0.0],[11592.0],[0.0],[1680.0],[32928.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
2,['Merced'],2020-06-10 to 2020-06-16,1,[63480.0],[2894000.0],[19000.0],[802550.0],[2072000.0],[246960.0],[1890.0],...,[0.0],[0.0],[2800.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
3,['Mono'],2020-06-10 to 2020-06-16,1,[5900.0],[461000.0],[20500.0],[19558.0],[381500.0],[308420.0],[30622.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
4,['Napa'],2020-06-10 to 2020-06-16,1,[78344.0],[774000.0],[612000.0],[172480.0],[660000.0],[820628.0],[1019012.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[36400.0],[0.0],[0.0],[0.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,[Merced],2021-03-13 to 2021-03-19,4,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
0,[Glenn],2021-03-13 to 2021-03-19,4,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
0,[Plumas],2021-03-13 to 2021-03-19,4,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
0,[Sierra],2021-03-13 to 2021-03-19,4,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]


In [36]:
# export as csv for use elsewhere
updated_df.to_csv(f"resources/ppe_crunched_{d_today}.csv", index=False, header=True)

In [37]:
# redundant backup
weekly_df.to_csv(f"resources/weekly_df_backup_{d_today}.csv", index=False, header=True)